In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint = "/workspace/runpod-playground/models/Mixtral-8x7B-Instruct-v0.1"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint, device_map="auto", load_in_4bit=False
)

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful virtual assistant trained by OpenAI.",
    },
    {"role": "user", "content": "Who are you?"},
]
tokenized_chat = tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True, return_tensors="pt"
).to(device)
print(tokenizer.decode(tokenized_chat[0]))

In [ ]:
outputs = model.generate(tokenized_chat, max_new_tokens=128)

In [ ]:
pipe = pipeline(
    "text-generation",
    checkpoint,
    device=device,
    use_fast=True,
    torch_dtype=torch.bfloat16,
    # trust_remote_code=True,
)

print(
    pipe(messages, max_new_tokens=128)[0]["generated_text"][-1]
)  # Print the assistant's response